# 코드 시작 전 설명
1. 음성 데이터에 대한 이론 및 과정을 잘 모르시는 분은 코드를 작성하기 전 overview를 한번 정독하시는 것을 권합니다.

2. 스켈레톤 코드가 적혀있는 해당 ipynb 파일을 다운받아 자신의 작업공간(코랩, 캐글노트북 등등)에서 코드를 작성합니다.

3. 각 코드에는 빈칸과 함께 구현 가이드 라인이 제공됩니다. 해당 가이드라인을 따라 코드를 작성하여 베이스 라인 성능을 달성합니다.
   진행에 어려움이 있으신 경우에는 feature 가공에 사용되는 함수의 documentation 설명을 읽으시는 것을 권합니다.
   
4. (선택)베이스라인에 도달하였다면, 음성 feature를 새롭게 가공하여 추가적인 성능 향상을 달성해봅니다.

In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/2022-ml-project3/train_data.csv
/kaggle/input/2022-ml-project3/test_data.csv
/kaggle/input/2022-ml-project3/sample_submit.csv
/kaggle/input/2022-ml-project3/test_data/test_data/test_132.wav
/kaggle/input/2022-ml-project3/test_data/test_data/test_185.wav
/kaggle/input/2022-ml-project3/test_data/test_data/test_268.wav
/kaggle/input/2022-ml-project3/test_data/test_data/test_344.wav
/kaggle/input/2022-ml-project3/test_data/test_data/test_093.wav
/kaggle/input/2022-ml-project3/test_data/test_data/test_131.wav
/kaggle/input/2022-ml-project3/test_data/test_data/test_327.wav
/kaggle/input/2022-ml-project3/test_data/test_data/test_120.wav
/kaggle/input/2022-ml-project3/test_data/test_data/test_064.wav
/kaggle/input/2022-ml-project3/test_data/test_data/test_371.wav
/kaggle/input/2022-ml-project3/test_data/test_data/test_028.wav
/kaggle/input/2022-ml-project3/test_data/test_data/test_250.wav
/kaggle/input/2022-ml-project3/test_data/test_data/test_408.wav
/kaggle/input/2022-ml-projec

In [2]:
import pandas as pd
import numpy as np
import sklearn
import os
from os.path import join

In [3]:
# DATA Load

DATA_PATH = "../input/2022-ml-project3"
# (참고) os.path.join 함수는 여러 문자열을 경로에 대한 문자열로 합쳐주는 함수입니다.
pd_train = pd.read_csv(join(DATA_PATH, 'train_data.csv'))
pd_test = pd.read_csv(join(DATA_PATH, 'test_data.csv'))

print(pd_train.info(), pd_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1008 entries, 0 to 1007
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ID         1008 non-null   int64 
 1   file_name  1008 non-null   object
 2   emotion    1008 non-null   object
dtypes: int64(1), object(2)
memory usage: 23.8+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 432 entries, 0 to 431
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ID         432 non-null    int64 
 1   file_name  432 non-null    object
dtypes: int64(1), object(1)
memory usage: 6.9+ KB
None None


# **Feature 추출**

* extract_feature 함수는 음성 데이터를 읽어서 다음의 과정들을 거치며 feature를 추출합니다.(함수(입력) --> 출력)
 1. Sampling&Quantization(continuos audio signal) --> Discrete audio signal
 2. Short Time Fourier Transfrom(Discrete audio signal) --> Spectrogram
 3. Mel-Filter(Spectrogram) --> Mel-Spectrogram
 4. Discrete Cosine Transform(Mel-Spectrogram) --> Mel Frequency Cepstrum Coefficient
 
위에 과정들은 모두 librosa 라이브러리에서 제공하는 함수들을 통하여 구현가능하며 사용되는 함수는 다음과 같습니다.

1. librosa.load : Continuos audio signal(.wav파일)을 Discrete audio signal로 읽음. 
2. librosa.stft : Discrte audio signal을 windowing하여 프레임별로 나눈 후 FFT(Fast Fourier Transform)을 수행
[stft_documentation](https://librosa.org/doc/latest/generated/librosa.stft.html?highlight=stft)
3. librosa.feature.melspectrogram : stft 함수를 통하여 구한 spectrogram을 입력으로 Mel-filter를 적용함으로써 Mel-Spectrogram 생성[melspectrogram_documentation](https://librosa.org/doc/latest/generated/librosa.feature.melspectrogram.html?highlight=melspectrogram)
4. librosa.feature.mfcc : melspectrogram 함수를 통하여 구한 Mel-spectrogram을 입력으로 DCT를 수행함으로써 MFCC를 생성 [mfcc_documentation](https://librosa.org/doc/latest/generated/librosa.feature.mfcc.html?highlight=mfcc)

In [4]:
# -------------------------------------
# extract_feature(file_name): <= 코드를 추가하여 음성 feature를 추출하는 코드를 완성하세요
# -------------------------------------
# 목적: MFCC를 비롯한 여러 음성 feature를 추출
# 입력인자: .wav 음성 파일의 이름
# 출력인자: 입력 파일을 가공한 feature들 (Spectrogram, Mel-spectrogram, MFCC)
# -------------------------------------

In [5]:
#----------step3. spectrogram을 구하세요.---------------------
# 구현 가이드 라인(3)
# ------------------------------------------------------------------------------
# 1. 입력 신호(X)를 librosa.stft 함수의 입력으로 하여 spectrogram을 구하세요.
#   - 참고) 사람의 음성은 20~40(ms) 사이 시간 내에서 현재 말하는 발음을 변경할 수 없다고 합니다.
#       시간축에 대한 구간을 나눌 때 20~40(ms) 이내 간격으로 나누기 위하여 n_fft 파라미터 값을 조정해주세요.(베이스라인 성능은 23ms 간격으로 나누었습니다.)
#       정확한 조정을 위하여 librosa documentation에서 librosa.stft 함수 내 n_fft 파라미터 설명을 참조하세요.
#
# 2. spectrogram에 절대값을 취하여 복소수 형태의 값을 바꾸세요.
#
# 3. 구한 spectrogram을 학습에 사용하기 위하여 프레임 축의 평균값을 취한 뒤 spectrogram_feature에 저장해주세요
#   -참고) spectrogram의 shape은 (frequency의 길이, 프레임 수)로 이루어져있습니다.
#       (np.mean함수를 사용하여 spectrogram의 shape이 (1, frequency의 길이)가 되면 성공.)
# -----------------------------------------------------------------------------

In [6]:
#-------------------------------------------------------------------------------
    
    #----------step4. Mel-spectrogram을 구하세요.---------------------
    # 구현 가이드 라인(4)
    # ------------------------------------------------------------------------------
    # 1. step3-2에서 구한 spectrogram을 제곱하여 power spectrogram을 만드세요.
    #
    # 2. power spectrogram을 librosa.feature.melspectrogram 함수의 입력으로 하여 mel-spectrogram을 구하세요.
    #   - 참고) documentation을 통해 librosa.feature.melspectrogram 함수의 입력 인자를 꼭 확인하셔서 올바르게 넣어주세요.
    #
    # 3. step4-2에서 구한 mel-spectrogram은 power-magnitude 값입니다. librosa.power_to_db함수를 통하여 power magnitude를 데쉬벨(db)로 변환하세요.
    #
    # 4. 구한 mel-spectrogram을 학습에 사용하기 위하여 프레임 축의 평균값을 취한 뒤 mel_spectrogram_feature에 저장해주세요
    #   -참고) mel-spectrogram의 shape은 (mel filter의 길이, 프레임 수)로 이루어져있습니다.
    #       (np.mean함수를 사용하여 mel-spectrogram의 shape이 (1, mel filter의 길이)가 되면 성공.)
    # -----------------------------------------------------------------------------

In [7]:
    #-------------------------------------------------------------------------------

    #----------step5. MFCC를 구하세요.---------------------
    # 구현 가이드 라인(5)
    # ------------------------------------------------------------------------------ 
    # 1. step4-3에서 데쉬벨로 변환한 mel-spectrogram을 librosa.feature.mfcc 함수의 입력으로 하여 MFCC를 구하세요.
    #   - 참고) documentation을 통해 librosa.feature.mfcc 함수의 입력 인자를 꼭 확인하셔서 올바르게 넣어주세요.
    #
    # 2. 구한 MFCC 학습에 사용하기 위하여 프레임 축의 평균값을 취한 뒤 mfcc_feature에 저장해주세요
    # -참고) MFCC shape은 (MFCC의 길이, 프레임 수)로 이루어져있습니다.
    #       (np.mean함수를 사용하여 MFCC의 shape이 (1, MFCC의 길이)가 되면 성공.)
    # -----------------------------------------------------------------------------

In [8]:
import librosa
import glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import librosa, librosa.display 


def extract_feature(file_name):
    # spectrogram
    result=np.array([])
    X, sample_rate = librosa.load(file_name, sr=22050) # 입력 신호(X)를 librosa.stft 함수의 입력으로 하여 spectrogram 구하기
    
    # - 참고) 사람의 음성은 20~40(ms) 사이 시간 내에서 현재 말하는 발음을 변경할 수 없다고 합니다.
    # 시간축에 대한 구간을 나눌 때 20~40(ms) 이내 간격으로 나누기 위하여 n_fft 파라미터 값을 조정해주세요.(베이스라인 성능은 23ms 간격으로 나누었습니다.)
    spectrogram = np.abs(librosa.stft(y = X, n_fft = 512)) # spectrogram에 절대값을 취하여 복소수 형태의 값 바꾸기
    spectrogram_feature = np.mean(spectrogram, axis = 1) # 프레임 축의 평균값을 취한 뒤 spectrogram_feature에 저장
    
    
    # Mel-spectrogram
    power_spectrogram = spectrogram**2 # pectrogram을 제곱하여 power spectrogram 구하기
    mel_spectrogram = librosa.feature.melspectrogram(S = power_spectrogram) # power spectrogram을 melspectrogram 함수의 입력으로 해 mel-spectrogram 구하기
    power_magnitude_to_db = librosa.power_to_db(S = mel_spectrogram) # librosa.power_to_db함수를 통하여 power magnitude를 데쉬벨(db)로 변환
    mel_spectrogram_feature= np.mean(power_magnitude_to_db, axis = 1) # 프레임 축의 평균값을 취한 뒤 mel_spectrogram_feature에 저장

    # MFCC
    MFCC = librosa.feature.mfcc(S = power_magnitude_to_db) # 데쉬벨로 변환한 mel-spectrogram을 librosa.feature.mfcc 함수의 입력으로 하여 MFCC를 구하기
    mfcc_feature = np.mean(MFCC, axis = 1) # 프레임 축의 평균값을 취한 뒤 mfcc_feature에 저장

    return spectrogram_feature, mel_spectrogram_feature, mfcc_feature 

# **데이터 불러오기**

### csv파일에 저장된 파일 이름과 학습용 label을 로드하여 학습 및 평가용 데이터를 불러오세요.

In [9]:
#DataFlair - Load the data and extract features for each sound file
# (참고) tqdm 은 진행률에 대한 프로그레스바를 알려주는 라이브러리입니다. 
# for문에 자주 사용되며, 얼마나 진행되었는지를 시각적으로 확인할 수 있어 자주 사용됩니다.

In [10]:
# ------------- step1. 학습용 데이터로더 코드를 작성하세요.----------------------
# 구현 가이드라인  (1)
# ------------------------------------------------------------
# train.csv 파일에 있는 음성 파일의 이름과 emotion 정보를 통하여 학습용 데이터를 로드하세요.
# 음성 파일의 정확한 경로 명을 extract_feature 함수의 입력으로 넣을 수 있게 경로를 잘 설정해보세요.
# extract_feature를 통해 구한 음성 feature들을 train_data 사전 속 배열에 알맞게 append 해주세요. ex) train_data['spectrogram'].append(spectrogram_feature)
# ------------------------------------------------------------
# 구현 가이드라인을 참고하여 코드를 작성해보세요.

# import pdb;pdb.set_trace() 를 통한 중단점을 찍어가며 가이드라인을 완성하는 것을 권장합니다.
# (힌트) os.path.join 함수를 이용하여 file_name(.wav 파일) 이 있는 경로를 맞춰주어야 합니다
# -------------step2. 평가용 데이터로더 코드를 작성하세요.-----------------
# 구현 가이드라인  (2)
# ------------------------------------------------------------
# test.csv 파일에 있는 음성 파일의 이름정보를 통하여 평가용 데이터를 로드하세요.
# 음성 파일의 정확한 경로 명을 extract_feature 함수의 입력으로 넣을 수 있게 경로를 잘 설정해보세요.
# extract_feature를 통해 구한 음성 feature들을 test_data 사전 속 배열에 알맞게 append 해주세요. ex) test_data['spectrogram'].append(spectrogram_feature)
# ------------------------------------------------------------
# 구현 가이드라인을 참고하여 코드를 작성해보세요.

In [11]:
from tqdm.notebook import tqdm
def load_data(data_info, isTrain=True):
    PATH = join('/kaggle','input','2022-ml-project3')
    if isTrain:
        train_data = {'spectrogram':[],'mel':[],'mfcc':[]} #음성 feature들을 담는 dictionary
        train_label = [] #학습에 사용할 label을 담는 list
        
        file_list = data_info['file_name']
        emotion_list = data_info['emotion']

        for file_name, emotion in tqdm(zip(file_list, emotion_list)): # zip을 통해 2개 이상의 변수를 한 번에 넘길 수 있다.
            
            # train.csv 파일에 있는 음성 파일의 이름과 emotion 정보를 통하여 학습용 데이터를 로드
            path = join(PATH,'train_data/train_data', file_name)
            train_label.append(emotion)
            
            # 음성 파일의 정확한 경로 명을 extract_feature 함수의 입력으로 넣음
            spectrogram_feature, mel_spectrogram_feature, mfcc_feature  = extract_feature(path) 
            
            # extract_feature를 통해 구한 음성 feature들을 train_data 사전 속 배열에 알맞게 append
            train_data['spectrogram'].append(spectrogram_feature)
            train_data['mel'].append(mel_spectrogram_feature)
            train_data['mfcc'].append(mfcc_feature)
            
        return train_data, np.array(train_label)
    
    else:
        test_data = {'spectrogram':[],'mel':[],'mfcc':[]} #음성 feature들을 담는 dictionary
        file_list = data_info['file_name']
    
        for file_name in tqdm(file_list):

            # test.csv 파일에 있는 음성 파일의 이름정보를 통하여 평가용 데이터를 로드
            path = join(PATH,'test_data/test_data', file_name)
            
            # 음성 파일의 정확한 경로 명을 extract_feature 함수의 입력으로 넣음
            spectrogram_feature, mel_spectrogram_feature, mfcc_feature  = extract_feature(path)
            
            # extract_feature를 통해 구한 음성 feature들을 test_data 사전 속 배열에 알맞게 append
            test_data['spectrogram'].append(spectrogram_feature)
            test_data['mel'].append(mel_spectrogram_feature)
            test_data['mfcc'].append(mfcc_feature)
            
        return test_data

#DataFlair - Split the dataset
train_data, y_train = load_data(pd_train)
test_data = load_data(pd_test, isTrain=False)

0it [00:00, ?it/s]

  0%|          | 0/432 [00:00<?, ?it/s]

# 모델 학습 및 추론
위에서 우리는 
1. 음성 신호에 대하여 Short Time Fourier Transform(stft)를 통해 **spectrogram**을 구하고,
2. 구한 spectrogram에 mel-filter를 씌워 **mel-spectrogram**을 구했으며,
3. 마지막으로 mel-spectrogram에 Discrete cosine transform(DCT) 취해줌으로써 **mfcc**까지 계산하였습니다.

위에서 구한 feature들은 모두 음성 데이터를 활용하는 다양한 작업(사람 음성 분류, 음성을 통한 감정 분류 등)에서 모델 학습에 사용할 수 있는 feature들입니다.
각각의 **feature들에 따른 모델의 정확도가 얼마나 차이**나는지를 확인해봅시다.

- 베이스라인의 분류기는 **RandomForestClassifier**를 사용합니다.
- **random_state에 따른 성능 차이가 발생하오니 반드시 random_state를 1로 고정해주시길 바랍니다.**

In [12]:
# ------------- step6. 음성 feature들을 가지고 모델을 학습하세요.----------------------
# 구현 가이드라인  (6)
# ------------------------------------------------------------
# dictionary 형태의 train_data 변수 내에는 spectrogram, mel-spectrogram, mfcc feature들이 존재합니다.
# 반복문을 통해 각 종류의 feature를 하나씩 불러오세요. ex) x_train = np.array(train_data[feature_name])
# 불러온 feature로 모델을 학습 및 추론 후 sample submit파일에 저장하세요.
# ------------------------------------------------------------
# 구현 가이드라인을 참고하여 코드를 작성해보세요.

In [13]:
#RandomForestClassifier로 음성 감정 분류 학습 및 평가
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(random_state = 1) # baseline : 0.57870

#SVC로 음성 감정 분류 학습 및 평가
from sklearn.svm import SVC

classifier = SVC(random_state = 1, C = 100, gamma = 'auto', tol=0.4, probability = True) # 0.66666
classifier = SVC(random_state = 1, C = 100, gamma = 'auto', tol=0.45, probability = True) # 0.67129 -> best!
classifier = SVC(random_state = 1, C = 100, gamma = 'auto', tol=0.5, probability = True) # 0.67129 -> best!
classifier = SVC(random_state = 1, C = 100, gamma = 'auto', tol=0.55, probability = True) # 0.65972


for feature_name in train_data.keys():
    # train_data에 있는 각 feature load
    
    # 해당 feature의 train_data, test_data load
    x_train = np.array(train_data[feature_name])
    x_test = np.array(test_data[feature_name])
    
    # StandardScaler로 표준화
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)

    # classifier 학습
    classifier.fit(x_train, y_train)
    predict = classifier.predict(x_test)    
    
    #Sample submit file 저장
    sample = pd.read_csv(join(DATA_PATH,'sample_submit.csv'))
    sample['emotion'] = predict.reshape(-1,1)
    sample.to_csv(feature_name+'.csv', index=False, header = True)
    if feature_name == 'mfcc': sample.to_csv('submission.csv',index=False,header=True)


In [14]:
train_data['spectrogram'][0].shape

(257,)

In [15]:
len(train_data['spectrogram'])

1008

In [16]:
np.array(train_data['spectrogram']).shape

(1008, 257)

In [17]:
np.array(train_data['spectrogram']).reshape(len(y_train), 257).shape

(1008, 257)

In [18]:
y_train.shape

(1008,)

(1008, 1, 257) -> (1008, 257)